In [1]:
import requests
from telegram import Bot, Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import CommandHandler, JobQueue, Application, CallbackQueryHandler, ContextTypes, CommandHandler, Updater
import datetime
import pandas as pd
import io

In [2]:
TELEGRAM_TOKEN = "6649758324:AAGNvtZ6e4CTaEPwACa9o3IzUcXzmN7zZz0"
CHAT_ID = "@break_vol_binance"


bot = Bot(token=TELEGRAM_TOKEN)
await bot.initialize()

BASE_URL = "https://api.binance.com/api/v3"


In [3]:
def get_symbol_data(symbol):
    url = f"{BASE_URL}/klines"
    params = {
        "symbol": symbol,
        "interval": "1h",
        "limit": 1
    }
    response = requests.get(url, params=params)
    data = response.json()
    columns = [
        "Kline open time",
        "Open price",
        "High price",
        "Low price",
        "Close price",
        "Volume",
        "Kline Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Unused field"
    ]
    df = pd.DataFrame(data, columns=columns)
    df.round(2)
    return df

def get_usdt_trading_pairs():
    response = requests.get("https://api.binance.com/api/v3/exchangeInfo")
    data = response.json()

    usdt_pairs = []

    for symbol_info in data['symbols']:
        if symbol_info['quoteAsset'] == 'USDT':
            usdt_pairs.append(symbol_info['symbol'])

    return usdt_pairs

def calculate_rsi(data, period=14):
    df = pd.DataFrame(data, columns=["Close"])

    df["Price Change"] = df["Close"].diff()

    df["Gain"] = df["Price Change"].apply(lambda x: x if x > 0 else 0)
    df["Loss"] = df["Price Change"].apply(lambda x: -x if x < 0 else 0)

    avg_gain = df["Gain"].rolling(window=period).mean()
    avg_loss = df["Loss"].rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    current_rsi = rsi.iloc[-1]
    return current_rsi

# get MA20 volume
def get_ma20_volume(symbol):
    url = f"{BASE_URL}/klines"
    params = {
        "symbol": symbol,
        "interval": "1h",
        "limit": 20
    }
    response = requests.get(url, params=params)
    data = response.json()
    volumes = [float(entry[5]) for entry in data]
    closing_prices = [float(entry[4]) for entry in data]
    current_rsi = calculate_rsi(closing_prices)
    ma20_volume = sum(volumes) / len(volumes)
    return round(ma20_volume, 2), current_rsi


async def send_chart(chart_symbol, caption):
  print("Sending chart...")
#   query = update.callback_query
#   await query.answer()
  symbol = f"BINANCE:{chart_symbol}"
  # Get chart image using CHART-IMG API
  chart_api_key = "esFwC38dqg3anwgXuhbaB8UeGt2YWKHW7Lx1nm6Q"
  chart_url = "https://api.chart-img.com/v2/tradingview/advanced-chart"
  headers = {
  "x-api-key": chart_api_key,
  "content-type": "application/json"
  }
  chart_payload = {
  "theme": "dark",
  "interval": "1h",
  "symbol": symbol,
  "override": {
      "showStudyLastValue": True
  },
  "studies": [
      {
      "name": "Volume",
      "forceOverlay": False
      },
      {
        "name": "Ichimoku Cloud",
        "input": {
          "in_0": 9,
          "in_1": 26,
          "in_2": 52,
          "in_3": 26
        },
        "override": {
          "ConversionLine.visible": True,
          "ConversionLine.linewidth": 1,
          "ConversionLine.plottype": "line",
          "ConversionLine.color": "rgb(33,150,243)",
          "BaseLine.visible": True,
          "BaseLine.linewidth": 1,
          "BaseLine.plottype": "line",
          "BaseLine.color": "rgb(128,25,34)",
          "LaggingSpan.visible": True,
          "LaggingSpan.linewidth": 1,
          "LaggingSpan.plottype": "line",
          "LaggingSpan.color": "rgb(67,160,71)",
          "LeadingSpanA.visible": True,
          "LeadingSpanA.linewidth": 1,
          "LeadingSpanA.plottype": "line",
          "LeadingSpanA.color": "rgb(165,214,167)",
          "LeadingSpanB.visible": True,
          "LeadingSpanB.linewidth": 1,
          "LeadingSpanB.plottype": "line",
          "LeadingSpanB.color": "rgb(250,161,164)",
          "Plots Background.visible": True,
          "Plots Background.transparency": 90
        }
      }
  ]
  }
  chart_response = requests.post(chart_url, headers=headers, json=chart_payload)
  image_buffer = io.BytesIO(chart_response.content)

  await bot.send_photo(chat_id= CHAT_ID,photo=image_buffer, caption=caption)


# Check trading conditions and send message to Telegram
tokens_to_check = ["BTCUSDT", "UNIUSDT", "MAVUSDT", "OPUSDT","UNFIUSDT","DOGEUSDT",
                    "SUIUSDT","BAKEUSDT","AUDIOUSDT","GRTUSDT","LITUSDT","BELUSDT","RENUSDT"]
async def check_conditions_and_send_message(context):
    print("Checking conditions...")
    for symbol in tokens_to_check:
        symbol_data = get_symbol_data(symbol)
        current_price = float(symbol_data['Close price'])
        open_price = float(symbol_data["Open price"])
        current_volume = float(symbol_data["Volume"])
        ma20_volume = get_ma20_volume(symbol)

        if current_price > open_price and current_volume > ma20_volume:
            keyboard = [[InlineKeyboardButton(text=f"View Chart {symbol}", callback_data=symbol)]]
            reply_markup = InlineKeyboardMarkup(keyboard)
            message = f"🚀BUY SIGNAL {symbol} 📈\n💸Price: {current_price}\nOpen Price: {open_price}\nVolume: {current_volume}\nMA20 Volume: {ma20_volume}"
            # await bot.send_message(chat_id=CHAT_ID, text=message, reply_markup=reply_markup)
            await send_chart(symbol, message)


In [19]:
def get_symbol_history(symbol, limit = 100):
    url = f"{BASE_URL}/klines"
    params = {
        "symbol": symbol,
        "interval": "1h",
        "limit": limit
    }
    response = requests.get(url, params=params)
    data = response.json()
    volumes = [float(entry[5]) for entry in data]
    closing_prices = [float(entry[4]) for entry in data]
    return volumes, closing_prices

In [20]:
a, b = get_symbol_history("BTCUSDT")

In [8]:
def calculate_last_ma(data, length):
    if len(data) < length:
        return None  # Not enough data to calculate MA

    last_ma = sum(data[-length:]) / length
    return last_ma


In [13]:
def ma_crossover_signal(data):
    if len(data) < 8:
        return None  # Not enough data to calculate MA crossovers

    ma5 = calculate_last_ma(data, 5)
    ma8 = calculate_last_ma(data, 8)

    if ma8 is None or ma5 is None:
        return None  # Not enough data to calculate MA crossovers

    if ma5 > ma8:
        return True
    else:
        return False

In [24]:
ma_crossover_signal(b)

False

In [27]:
def calculate_ema(data, period):
    if len(data) < period:
        return None  # Not enough data to calculate EMA

    smoothing_factor = 2 / (period + 1)
    ema = data[-1]  # Initialize EMA with the first data point

    for i in range(len(data) - period, len(data)):
        ema = (data[i] - ema) * smoothing_factor + ema

    return ema

def macd_crossover_signal(data):
    if len(data) < 26:
        return None  # Not enough data to calculate MACD

    ema_12 = calculate_ema(data, 12)
    ema_26 = calculate_ema(data, 26)

    if ema_12 is None or ema_26 is None:
        return None  # Not enough data to calculate MACD

    macd = ema_12 - ema_26
    if macd > 0:
        return True
    else:
        return False
        

In [28]:
macd_crossover_signal(b)

False

In [30]:
def calculate_ichimoku_cloud(data):
    if len(data) < 52:
        return None  # Not enough data to calculate Ichimoku Cloud

    # Calculate Tenkan Sen (Conversion Line)
    tenkan_high = max(data[-9:])  # Highest high of the last 9 periods
    tenkan_low = min(data[-9:])  # Lowest low of the last 9 periods
    tenkan_sen = (tenkan_high + tenkan_low) / 2

    # Calculate Kijun Sen (Base Line)
    kijun_high = max(data[-26:])  # Highest high of the last 26 periods
    kijun_low = min(data[-26:])  # Lowest low of the last 26 periods
    kijun_sen = (kijun_high + kijun_low) / 2

    # Calculate Senkou Span A (Leading Span A)
    senkou_span_a = (tenkan_sen + kijun_sen) / 2

    # Calculate Senkou Span B (Leading Span B)
    kumo_high = max(data[-52:])  # Highest high of the last 52 periods
    kumo_low = min(data[-52:])  # Lowest low of the last 52 periods
    senkou_span_b = (kumo_high + kumo_low) / 2

    return senkou_span_a, senkou_span_b

def is_price_under_cloud(data):
    ichimoku_cloud_data = calculate_ichimoku_cloud(data)
    
    if ichimoku_cloud_data is None:
        return None  # Not enough data to determine
    
    senkou_span_a, senkou_span_b = ichimoku_cloud_data
    
    if data[-1] < senkou_span_a and data[-1] < senkou_span_b:
        return True  # Price is under the cloud
    else:
        return False  # Price is not under the cloud

# Example usage
data = [110, 112, 114, 116, 118, 120, 118, 116, 114, 112, 110, 108, 106, 104, 102, 100, 98, 96, 94, 92, 90, 88, 86, 84, 82, 80, 78, 76, 74, 72, 70, 68, 66]

is_under_cloud = is_price_under_cloud(b)
print("Price is under cloud:", is_under_cloud)


Price is under cloud: True


In [ ]:
async def check_conditions_and_send_message(context):
    print("Checking conditions...")
    for symbol in tokens_to_check:
        symbol_data = get_symbol_data(symbol)
        current_price = float(symbol_data['Close price'])
        open_price = float(symbol_data["Open price"])
        current_volume = float(symbol_data["Volume"])
        ma20_volume = get_ma20_volume(symbol)
        macd_value = calculate_macd(closing_prices)  # Assuming you have a list of closing prices
        ichimoku_data = calculate_ichimoku_cloud(closing_prices)  # Assuming you have a list of closing prices for Ichimoku

        conditions = {
            "cross_ma": current_price > open_price and current_volume > ma20_volume,
            "macd": macd_value is not None and macd_value > 0,  # Modify this condition as needed
            "break_volume": current_volume > ma20_volume * 1.5,  # Adjust this threshold as needed
            "under_cloud": ichimoku_data is not None and is_price_under_cloud(closing_prices)
        }

        signal_strength = sum(conditions.values())

        if signal_strength == 4:  # All conditions are true
            signal_text = "Strong buy"
        elif signal_strength >= 2:  # At least two conditions are true
            signal_text = "Medium"
        elif signal_strength >= 1:  # At least one condition is true
            signal_text = "Low signal"
        else:
            signal_text = "No Signal"

        if signal_text != "No Signal":
            keyboard = [[InlineKeyboardButton(text=f"View Chart {symbol}", callback_data=symbol)]]
            reply_markup = InlineKeyboardMarkup(keyboard)
            message = f"🚀{signal_text.upper()} {symbol} 📈\n💸Price: {current_price}\nOpen Price: {open_price}\nVolume: {current_volume}\nMA20 Volume: {ma20_volume}\nMACD: {macd_value}"
            await send_chart(symbol, message)


In [4]:
current_time = datetime.datetime.now()
next_hour = current_time.replace(minute=0, second=0, microsecond=0) + datetime.timedelta(minutes=45)
time_to_wait = (next_hour - current_time).total_seconds()
print("Waiting for next hour to start. Time to wait: ", time_to_wait)

Waiting for next hour to start. Time to wait:  2633.16711


In [5]:
application = Application.builder().token(TELEGRAM_TOKEN).build()
# application.job_queue.run_repeating(check_conditions_and_send_message, interval=3600 , first= time_to_wait)
# application.add_handler(CommandHandler('start_check', check_conditions_and_send_message))
# application.add_handler(CallbackQueryHandler(send_chart))